In [2]:
suppressMessages(library(Seurat))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(viridis))
suppressMessages(library(ggplot2))
options(warn=-1)

In [1]:
set.seed(42)
getwd()

[1] "/data/scATA/SingleCellOpenChromatin/local/VisiumHeartRevision/IntegrativeAnalysis/Lymphoid/code"

In [5]:
## find cell type names
obj <- readRDS(glue::glue("../../../visiumSpatial/from_rico/processed_visium/objects/AKK001_157785.rds"))
DefaultAssay(obj) <- "c2l_props"
rownames(obj)

[1] "Adipo"    "CM"       "Endo"     "Fib"      "Lymphoid" "Mast"    
 [7] "Myeloid"  "Neuronal" "PC"       "prolif"   "vSMCs"

In [2]:
resolutions <- seq(0.2, 0.9, 0.1)

In [3]:
# obj <- readRDS(glue::glue("../../../visiumSpatial/from_rico/processed_visium/objects/AKK004_157772.rds"))

# obj

# all.markers <- readRDS(glue::glue("../data/coembed/MarkerGenes/res.0.2.rds"))
# gene.use <- intersect(rownames(obj), rownames(all.markers))
# all.markers <- all.markers[gene.use, ]

# all.markers <- all.markers %>%
#     subset(p_val_adj < 0.05) %>%
#     group_by(cluster) %>%
#     top_n(n = 50, wt = avg_log2FC) %>%
#     subset(., select = c("cluster", "gene"))

# all.markers <- split(all.markers, all.markers$cluster)

# for(cl in names(all.markers)){
#    all.markers[[cl]] <- all.markers[[cl]]$gene 
# }

# DefaultAssay(obj) <- "c2l_props"
# p1 <- SpatialFeaturePlot(obj, features = "Fib") +
# scale_fill_viridis(option = "D")

# DefaultAssay(obj) <- "c2l"
# p2 <- SpatialFeaturePlot(obj, features = "Fib") +
# scale_fill_viridis(option = "D")

# options(repr.plot.height = 5, repr.plot.width = 10)

# p1 + p2

# fib_score <- obj@assays$c2l_props@data["Fib", ]

# DefaultAssay(obj) <- "SCT"
# obj <- AddModuleScore(obj, features = all.markers)

# ## multiply the module score with cell2location deconvolution score
# # get module score
# df_module_score <- as.data.frame(obj@meta.data)%>%
#     select(., contains("Cluster"))

# df_module_score <- df_module_score * fib_score

# obj <- AddMetaData(obj, metadata = df_module_score)

# min_score <- min(df_module_score)
# max_score <- max(df_module_score)

# plotlist <- SpatialFeaturePlot(obj, features = c("Cluster1", "Cluster2", "Cluster3", "Cluster4", "Cluster5"), combine = FALSE)

# plotlist <- lapply(plotlist, function(p){
#     p <- p + scale_fill_viridis(limits = c(min_score, max_score))
# })

# p <- patchwork::wrap_plots(plotlist, nrow = 2)

# options(repr.plot.height = 10, repr.plot.width = 20)

# p

In [4]:
df_anno <- read.csv("../../../visiumSpatial/metadata/sample_annotation.csv", header = TRUE)

patient_region_id <- df_anno$patient_region_id
patient_region_id <- stringr::str_replace_all(patient_region_id, "/", "_")

names(patient_region_id) <- df_anno$sample_id

In [5]:
for(res in resolutions){
    if(!dir.exists(glue::glue("../data/visium/cell_states_mapping/res_{res}"))){
        dir.create(glue::glue("../data/visium/cell_states_mapping/res_{res}"), recursive = TRUE)
    }
    
    for(sample in names(patient_region_id)){
        message(glue::glue("processing sample: {sample}"))

        # load visium object
        obj <- readRDS(glue::glue("../../../visiumSpatial/from_rico/processed_visium/objects/{sample}.rds"))
    
        DefaultAssay(obj) <- "c2l_props"
        p1 <- SpatialFeaturePlot(obj, features = "Myeloid") + ggtitle("c2l: proportion")
   
        DefaultAssay(obj) <- "c2l"
        p2 <- SpatialFeaturePlot(obj, features = "Myeloid") + ggtitle("c2l: count")

        DefaultAssay(obj) <- "SCT"
        
        # load marker genes
        all.markers <- readRDS(glue::glue("../data/coembed/MarkerGenes/res.{res}.rds"))
        
        # only keep genes that are found in spatial dta
        gene.use <- intersect(rownames(obj), rownames(all.markers))
        all.markers <- all.markers[gene.use, ]
        
        # get number of clusters
        n_clusters <- length(unique(all.markers$cluster))
        
        # get the marker genes for each cluster
        all.markers <- all.markers %>%
            subset(p_val_adj < 0.05) %>%
            #group_by(cluster) %>%
            #top_n(n = 50, wt = avg_log2FC) %>%
            subset(., select = c("cluster", "gene"))
        
        # convert dataframe to list for module score calculation
        all.markers <- split(all.markers, all.markers$cluster)
        for(cl in names(all.markers)){
           all.markers[[cl]] <- all.markers[[cl]]$gene 
        }
        
        # get module score
        obj <- AddModuleScore(obj, features = all.markers, assay = "SCT")
        
        df_module_score <- as.data.frame(obj@meta.data)%>%
            select(., contains("Cluster"))


        # plot module score after normalization by proportion from cell2location
        fib_score_prop <- obj@assays$c2l_props@data["Myeloid", ]
        df_module_score_prop <- df_module_score * fib_score_prop
        
        obj <- AddMetaData(obj, metadata = df_module_score_prop)
        min_score <- min(df_module_score_prop)
        max_score <- max(df_module_score_prop)
        features <- paste0("Cluster", 1:n_clusters)
        
        plotlist <- SpatialFeaturePlot(obj, features = features, combine = FALSE)
        plotlist <- lapply(1:length(plotlist), function(i){
            cl <- i - 1
           p <- plotlist[[i]] + ggtitle(glue::glue("Cluster{cl}")) + theme(legend.title = element_blank())
           return(p)
        })
        ps1 <- patchwork::wrap_plots(plotlist, nrow = 2) +
            patchwork::plot_annotation(title = "Normalized by proportion")

        # plot module score after normalization by count from cell2location
        fib_score_count <- obj@assays$c2l@data["Myeloid", ]
        df_module_score_count <- df_module_score * fib_score_count
        obj <- AddMetaData(obj, metadata = df_module_score_count)
        min_score <- min(df_module_score_count)
        max_score <- max(df_module_score_count)
        features <- paste0("Cluster", 1:n_clusters)
        
        plotlist <- SpatialFeaturePlot(obj, features = features, combine = FALSE)
        plotlist <- lapply(1:length(plotlist), function(i){
            cl <- i - 1
           p <- plotlist[[i]] + ggtitle(glue::glue("Cluster{cl}")) + theme(legend.title = element_blank())
           return(p)
        })
        ps2 <- patchwork::wrap_plots(plotlist, nrow = 2) +
            patchwork::plot_annotation(title = "Normalized by count")
        
        
        patient_region <- patient_region_id[sample]

        pdf(glue::glue("../data/visium/cell_states_mapping/res_{res}/{sample}_{patient_region}.pdf"), width = 20, height = 10)
        print(p1+p2)
        print(ps1)
        print(ps2)
        dev.off()
    }
}

processing sample: AKK006_157771

processing sample: Visium_1_CK279

processing sample: Visium_3_CK281

processing sample: Visium_4_CK282

processing sample: Visium_12_CK290

processing sample: Visium_13_CK291

processing sample: Visium_16_CK294

processing sample: Visium_14_CK292

processing sample: AKK004_157772

processing sample: Visium_11_CK289

processing sample: Visium_18_CK296

processing sample: Visium_9_CK287

processing sample: Visium_19_CK297

processing sample: Visium_7_CK285

processing sample: Visium_20_CK298

processing sample: Visium_15_CK293

processing sample: AKK002_157779

processing sample: AKK003_157775

processing sample: Visium_8_CK286

processing sample: AKK002_157782

processing sample: Visium_2_CK280

processing sample: Visium_17_CK295



In [ ]:
sessionInfo()